In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

data = pd.read_csv('assets/csv/reviews_dataset.csv')
bad_words = pd.read_csv('assets/csv/bad_words.csv')
amazon_reviews = pd.read_json('assets/json/Clothing_Shoes_and_Jewelry_5.json', lines=True)
amazon_reviews.dropna(inplace=True)
data = data.dropna()
bad_words = bad_words.dropna()
bad_words_rate = pd.DataFrame(np.zeros((bad_words.shape[0], 1)).astype(int))

amazon_texts = amazon_reviews['reviewText']

x = data['text']
x = pd.concat([x, bad_words['jigaboo'], amazon_texts], ignore_index=True)

y = data['rate']

print(x.shape)

accuracy = 4

(302482,)


In [21]:
amazon_rates = amazon_reviews['overall'].astype(int)
amazon_rates = amazon_rates.apply(lambda rate: 1 if rate > accuracy else 0)

In [3]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

x = x.apply(lambda x: " ".join(x.lower() for x in str(x).split()))


def contractions(s):
    s = re.sub(r"won’t", "will not ", s)
    s = re.sub(r"would’t", "would not ", s)
    s = re.sub(r"could’t", "could not ", s)
    s = re.sub(r"\’d", " would", s)
    s = re.sub(r"can\’t", "can not ", s)
    s = re.sub(r"n\’t", " not ", s)
    s = re.sub(r"\’re", " are", s)
    s = re.sub(r"\’s", " is ", s)
    s = re.sub(r"\’ll", " will", s)
    s = re.sub(r"\’t", " not ", s)
    s = re.sub(r"\’ve", " have", s)
    s = re.sub(r"\’m", " am", s)
    return s


x = x.apply(lambda text: contractions(text))

x = x.apply(lambda x: " ".join([re.sub('[^A-Za-z]+', '', x) for x in nltk.word_tokenize(x)]))
x = x.apply(lambda text: re.sub(' +', ' ', text))

stop = stopwords.words('english')
x = x.apply(lambda text: " ".join([text for text in text.split() if text not in stop]))

lemmatizer = WordNetLemmatizer()
x = x.apply(lambda text: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(text)]))

In [22]:
y = y.apply(lambda rate: 1 if rate > accuracy else 0)

y = pd.concat([y, bad_words_rate, amazon_rates])
y.reset_index(inplace=True)
y.drop(columns=['index'], inplace=True)

print(y.shape)

(302482, 1)


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
processed_x = vectorizer.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(processed_x, y)

In [24]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, solver='saga')

clf.fit(x_train, y_train)

y_test_pred = clf.predict(x_test)

C:\Users\Gustav\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_test_pred, output_dict=True)
print(report)

{'0': {'precision': 0.7922175275998364, 'recall': 0.7325603377654547, 'f1-score': 0.7612218839013849, 'support': 31738}, '1': {'precision': 0.8165668964622999, 'recall': 0.861039582526263, 'f1-score': 0.8382137628111272, 'support': 43883}, 'accuracy': 0.8071170706549768, 'macro avg': {'precision': 0.8043922120310681, 'recall': 0.7967999601458589, 'f1-score': 0.7997178233562561, 'support': 75621}, 'weighted avg': {'precision': 0.8063475094010752, 'recall': 0.8071170706549768, 'f1-score': 0.80590040735646, 'support': 75621}}


In [27]:
sample_review = pd.DataFrame(["The delivery was so late and my package was damaged i contacted theiuir support and they didn't respond complete trash doesn't recommend."])

sample_review = sample_review.apply(lambda x: " ".join(x.lower() for x in str(x).split()))
sample_review = sample_review.apply(lambda text: contractions(text))
sample_review = sample_review.apply(lambda x: " ".join([re.sub('[^A-Za-z]+', '', x) for x in nltk.word_tokenize(x)]))
sample_review = sample_review.apply(lambda text: re.sub(' +', ' ', text))
sample_review = sample_review.apply(lambda text: " ".join([text for text in text.split() if text not in stop]))
sample_review = sample_review.apply(lambda text: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(text)]))

vectorizer_2 = TfidfVectorizer(vocabulary=vectorizer.get_feature_names_out())

processed_text = vectorizer_2.fit_transform(sample_review)

y_test_pred = clf.predict(processed_text)
print('Predicted rate 1/0:\t', y_test_pred[0])

Predicted rate 1/0:	 0


In [28]:
output = pd.DataFrame()
output['review'] = x
output['rate'] = y
print(output)

                                                   review  rate
0       Absolutely wonderful - silky and sexy and comf...     0
1       Love this dress!  it's sooo pretty.  i happene...     1
2       I had such high hopes for this dress and reall...     0
3       I love, love, love this jumpsuit. it's fun, fl...     1
4       This shirt is very flattering to all due to th...     1
...                                                   ...   ...
302477  I don't normally go ga-ga over a product very ...     1
302478  I've been traveling back and forth to England ...     1
302479  These are very nice packing cubes and the 18 x...     1
302480  I am on vacation with my family of four and th...     1
302481  When I signed up to receive a free set of Shac...     1

[302482 rows x 2 columns]


In [29]:
output.to_csv('assets/csv/output.csv', index=False)